In [1]:
library(caTools)
library(C50)
library(randomForest)
library(caret)
library(e1071)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'ggplot2'

The following object is masked from 'package:randomForest':

    margin



In [2]:
data <- readRDS('data.rds')

In [3]:
data$Customer.Satisfaction[data$Likelihood.to.recommend>7] <- "High"
data$Customer.Satisfaction[data$Likelihood.to.recommend<=7 & data$Likelihood.to.recommend>4] <- "Med"
data$Customer.Satisfaction[data$Likelihood.to.recommend<=4] <- "Low"
data$Customer.Satisfaction <- factor(as.character(data$Customer.Satisfaction))

In [4]:
data$Destination.City <- NULL
data$Origin.City <- NULL
data$Year.of.First.Flight <- NULL
data$Partner.Code <- NULL
data$Scheduled.Departure.Hour <- NULL
data$olong <- NULL
data$olat <- NULL
data$dlong <- NULL
data$dlat <- NULL
data$Flight.date <- NULL
data$Likelihood.to.recommend <- NULL
data$Day.of.Month <- NULL

In [5]:
head(data)

Airline.Status,Age,Gender,Price.Sensitivity,Flights.Per.Year,Loyalty,Type.of.Travel,Total.Freq.Flyer.Accts,Shopping.Amount.at.Airport,Eating.and.Drinking.at.Airport,Class,Partner.Name,Origin.State,Destination.State,Departure.Delay.in.Minutes,Arrival.Delay.in.Minutes,Flight.cancelled,Flight.time.in.minutes,Flight.Distance,Customer.Satisfaction
Blue,30,F,3,6,0.0769,Business travel,0,15,30,Eco Plus,Sigma Airlines Inc.,Minnesota,Arizona,0,0,N,165,1276,Med
Silver,57,F,1,27,-0.6875,Personal Travel,0,0,65,Eco,Northwest Business Airlines Inc.,California,Utah,0,2,N,85,532,Med
Silver,75,F,3,35,-0.8421,Personal Travel,0,0,15,Eco,FlyHere Airways,Nevada,Georgia,5,0,N,191,1747,High
Blue,55,F,1,13,-0.6250,Business travel,0,10,35,Eco,Oursin Airlines Inc.,Louisiana,California,0,0,N,257,1911,High
Blue,80,M,0,77,-0.9494,Personal Travel,0,0,30,Eco,Sigma Airlines Inc.,Georgia,Florida,0,0,N,54,404,Low
Blue,48,F,2,52,-0.8909,Personal Travel,0,0,30,Eco,FlyFast Airways Inc.,Ohio,Illinois,0,13,N,72,315,Low


In [6]:
str(data)

'data.frame':	10268 obs. of  20 variables:
 $ Airline.Status                : Factor w/ 4 levels "Blue","Gold",..: 1 4 4 1 1 1 1 1 1 1 ...
 $ Age                           : int  30 57 75 55 80 48 68 31 17 41 ...
 $ Gender                        : Factor w/ 2 levels "F","M": 1 1 1 1 2 1 2 1 1 2 ...
 $ Price.Sensitivity             : int  3 1 3 1 0 2 1 1 2 1 ...
 $ Flights.Per.Year              : int  6 27 35 13 77 52 33 15 8 3 ...
 $ Loyalty                       : num  0.0769 -0.6875 -0.8421 -0.625 -0.9494 ...
 $ Type.of.Travel                : Factor w/ 3 levels "Business travel",..: 1 3 3 1 3 3 3 2 1 1 ...
 $ Total.Freq.Flyer.Accts        : int  0 0 0 0 0 0 0 2 1 0 ...
 $ Shopping.Amount.at.Airport    : int  15 0 0 10 0 0 0 10 30 0 ...
 $ Eating.and.Drinking.at.Airport: int  30 65 15 35 30 30 210 90 50 10 ...
 $ Class                         : Factor w/ 3 levels "Business","Eco",..: 3 2 2 2 2 2 3 2 2 2 ...
 $ Partner.Name                  : Factor w/ 14 levels "Cheapseats Airlines I

In [7]:
data1 <- sample.split(data,SplitRatio = 0.3)
train <- subset(data,data1==FALSE)
rownames(train) <- 1:nrow(train)
test <- subset(data,data1==TRUE)
row.names(test) <- 1:nrow(test)

In [8]:
tree_m <- C5.0(x=train[,-20],y=train$Customer.Satisfaction,trails=10)

In [9]:
summary(tree_m)


Call:
C5.0.default(x = train[, -20], y = train$Customer.Satisfaction, trails = 10)


C5.0 [Release 2.07 GPL Edition]  	Sun Dec 15 23:28:24 2019
-------------------------------

Class specified by attribute `outcome'

Read 7187 cases (20 attributes) from undefined.data

Decision tree:

Type.of.Travel = Personal Travel:
:...Airline.Status = Silver:
:   :...Departure.Delay.in.Minutes > 55: Med (32/2)
:   :   Departure.Delay.in.Minutes <= 55:
:   :   :...Eating.and.Drinking.at.Airport <= 51:
:   :       :...Flight.cancelled = Y: High (3)
:   :       :   Flight.cancelled = N:
:   :       :   :...Arrival.Delay.in.Minutes > 4: Med (26)
:   :       :       Arrival.Delay.in.Minutes <= 4:
:   :       :       :...Class in {Business,Eco}: High (51/19)
:   :       :           Class = Eco Plus: Med (5)
:   :       Eating.and.Drinking.at.Airport > 51:
:   :       :...Eating.and.Drinking.at.Airport > 200: Med (11/1)
:   :           Eating.and.Drinking.at.Airport <= 200:
:   :           :...Total.Freq

In [10]:
tree_p <- predict(tree_m,test[,-20])

In [11]:
confusionMatrix(tree_p,test$Customer.Satisfaction)

Confusion Matrix and Statistics

          Reference
Prediction High  Low  Med
      High 1570  102  400
      Low    29  249  103
      Med   159  153  316

Overall Statistics
                                          
               Accuracy : 0.693           
                 95% CI : (0.6763, 0.7092)
    No Information Rate : 0.5706          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4334          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: High Class: Low Class: Med
Sensitivity               0.8931    0.49405     0.3858
Specificity               0.6206    0.94878     0.8621
Pos Pred Value            0.7577    0.65354     0.5032
Neg Pred Value            0.8137    0.90556     0.7949
Prevalence                0.5706    0.16358     0.2658
Detection Rate            0.5096    0.08082     0.1026
Detection Prevalence      0.

In [ ]:
#obj <- tune.randomForest(Customer.Satisfaction ~ ., data = train, mtry=c(5,6,7,8,9,10) , ntree = c(500,600,700,800,900,100) )

In [12]:
tree_r <- randomForest(Customer.Satisfaction ~ ., data = train, importance = TRUE,mtry=8,ntree = 800)

In [13]:
tree_r_p <- predict(tree_r,test[,-20])

In [14]:
confusionMatrix(tree_r_p,test$Customer.Satisfaction)

Confusion Matrix and Statistics

          Reference
Prediction High  Low  Med
      High 1591   90  346
      Low    16  257  100
      Med   151  157  373

Overall Statistics
                                          
               Accuracy : 0.7209          
                 95% CI : (0.7047, 0.7367)
    No Information Rate : 0.5706          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4888          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: High Class: Low Class: Med
Sensitivity               0.9050    0.50992     0.4554
Specificity               0.6704    0.95499     0.8638
Pos Pred Value            0.7849    0.68901     0.5477
Neg Pred Value            0.8416    0.90879     0.8142
Prevalence                0.5706    0.16358     0.2658
Detection Rate            0.5164    0.08341     0.1211
Detection Prevalence      0.